## Imports

In [1]:
import dostools
import importlib
import numpy as np
import pickle
import torch
import sys
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import matplotlib
import time
torch.set_default_dtype(torch.float64) 
%matplotlib notebook
matplotlib.rcParams['figure.figsize'] = (10, 10)
sys.modules['dostools.src'] = dostools

In [2]:
import dostools.datasets.data as data
import dostools.utils.utils as utils

n_structures = 1039
np.random.seed(0)
n_train = int(0.8 * n_structures)
train_index = np.arange(n_structures)
np.random.shuffle(train_index)
test_index = train_index[n_train:]
train_index = train_index[:n_train]

with torch.no_grad():
    structures = data.load_structures(":")
    n_structures = len(structures) #total number of structures
    for structure in structures:#implement periodicity
        structure.wrap(eps = 1e-12) 
    n_atoms = np.zeros(n_structures, dtype = int) #stores number of atoms in each structures
    for i in range(n_structures):
        n_atoms[i] = len(structures[i])

    #eigenergies, emin, emax = dostools.src.datasets.data.load_eigenenergies(unpack = True, n_structures = len(structures))
    xdos = torch.tensor(data.load_xdos())
    ldos = torch.tensor(data.load_ldos())
    ldos *= 2

    print ("ldos shape is {}".format(ldos.shape))
    mean_dos_per_atom = ldos[train_index].mean(axis = 0) #only calculated for train set to prevent data leakage
    print ("mean dos shape is {}".format(mean_dos_per_atom.shape))
    
    
    y_pw = ldos - mean_dos_per_atom
    y_lcdf = torch.cumsum(y_pw, dim = 1)
    _, pc_vectors = utils.build_pc(ldos[train_index], mean_dos_per_atom[None,:], n_pc = 10)
    y_pc = utils.build_coeffs(ldos - mean_dos_per_atom[None,:], pc_vectors)
    Silicon = data.load_features()
    kMM = data.load_kMM()

ldos shape is torch.Size([1039, 778])
mean dos shape is torch.Size([778])
Variance covered with 10 PCs is = 0.9871211778950163


## Evaluator

In [3]:
import dostools.evaluation.evaluation as evaluation
importlib.reload(evaluation)
import dostools.models.training as training
importlib.reload(training)

targets = {
    'pw' : ldos,
    'lcdf' : y_lcdf,
    'pc' : y_pc
}
evaluator = evaluation.Evaluator(targets, xdos, mean_dos_per_atom)

In [4]:
evaluator.GetTargetRMSE(y_pw, "pw", train_index, test_index)

array([2.89371661e-15, 2.91656217e-15, 2.89787701e-15, 1.70171194e-17,
       1.77631607e-17, 1.71690667e-17])

## Dataset and DataLoader

In [14]:
import dostools.datasets.dataset as data
from torch.utils.data import TensorDataset, DataLoader
import dostools.consistency.consistency as consistency

device = 'cpu'
kwargs = {"pin_memory":True} if device == "cuda:0" else {}
#Dataset
y_shifted = copy.deepcopy(y_pw)
#y_shifted[train_index][:100] = consistency.shifted_ldos(y_shifted[:100], xdos, torch.zeros(100)-10)
train_data_soap = TensorDataset(Silicon.Features["structure_avedescriptors"][train_index].double(), y_shifted[train_index].double())
train_data_kernel = TensorDataset(Silicon.Features["structure_avekerneldescriptors"][train_index].double(), y_shifted[train_index].double())

test_data_soap = TensorDataset(Silicon.Features["structure_avedescriptors"][test_index].double(), y_shifted[test_index].double())
test_data_kernel = TensorDataset(Silicon.Features["structure_avekerneldescriptors"][test_index].double(), y_shifted[test_index].double())

#Dataloader

train_dataloader_soap = DataLoader(train_data_soap, batch_size = len(train_data_soap), shuffle = False, **kwargs)
train_dataloader_kernel = DataLoader(train_data_kernel, batch_size = len(train_data_kernel), shuffle = False, **kwargs)



In [5]:
import dostools.datasets.dataset as data
from torch.utils.data import TensorDataset, DataLoader
import dostools.consistency.consistency as consistency


soap_mean = torch.mean(Silicon.Features["structure_avedescriptors"][train_index], dim = 0)
soap_total = Silicon.Features["structure_avedescriptors"] - soap_mean

kernel_mean = torch.mean(Silicon.Features["structure_avekerneldescriptors"][train_index], dim = 0)
kernel_total = Silicon.Features["structure_avekerneldescriptors"] - kernel_mean


device = 'cpu'
kwargs = {"pin_memory":True} if device == "cuda:0" else {}
#Dataset
y_shifted = copy.deepcopy(y_pw)
#y_shifted[train_index][:100] = consistency.shifted_ldos(y_shifted[:100], xdos, torch.zeros(100)-10)
train_data_soap = TensorDataset(soap_total[train_index].double(), y_shifted[train_index].double())
train_data_kernel = TensorDataset(kernel_total[train_index].double(), y_shifted[train_index].double())

test_data_soap = TensorDataset(soap_total[test_index].double(), y_shifted[test_index].double())
test_data_kernel = TensorDataset(kernel_total[test_index].double(), y_shifted[test_index].double())

#Dataloader

train_dataloader_soap = DataLoader(train_data_soap, batch_size = 8, shuffle = False, **kwargs)
train_dataloader_kernel = DataLoader(train_data_kernel, batch_size = 8, shuffle = False, **kwargs)



## Loss Function

In [6]:
import dostools.src.consistency.consistency as consistency
import dostools.src.loss.loss as loss
importlib.reload(loss)
importlib.reload(consistency)

def t_get_mse(a, b, xdos = None, perc = False):
    if xdos is not None:
        mse = (torch.trapezoid((a - b)**2, xdos, axis=1)).mean()
        if not perc:
            return mse
        else:
            mean = b.mean(axis = 0)
            std = torch.sqrt(torch.trapezoid((b - mean)**2, xdos, axis=1)).mean()
            return (100 * mse / std)
    else:
        mse = ((a - b)**2).mean(dim = 1)
        if len(mse.shape) > 1:
            raise ValueError('Loss became 2D')
        if not perc:
            return torch.mean(mse, 0)
        else:
            return torch.mean(100 * (mse / b.std(dim=0, unbiased = True)),0)


## Evaluation Function

In [7]:
def t_get_rmse(a, b, xdos=None, perc=False): #account for the fact that DOS is continuous but we are training them pointwise
    """ computes  Root Mean Squared Error (RMSE) of array properties (DOS/aofd).
         a=pred, b=target, xdos, perc: if False return RMSE else return %RMSE"""
    #MIGHT NOT WORK FOR PC
    if xdos is not None:
        rmse = torch.sqrt(torch.trapezoid((a - b)**2, xdos, axis=1)).mean()
        if not perc:
            return rmse
        else:
            mean = b.mean(axis = 0)
            std = torch.sqrt(torch.trapezoid((b - mean)**2, xdos, axis=1)).mean()
            return (100 * rmse / std)
    else:
        rmse = torch.sqrt(((a - b)**2).mean(dim =0))
        if not perc:
            return torch.mean(rmse, 0)
        else:
            return torch.mean(100 * (rmse / b.std(dim = 0,unbiased=True)), 0)

## Linear Model

In [8]:
import dostools.src.models.models as models
import dostools.src.models.training as training
import dostools.src.models.architectures as architecture
import dostools.src.loss.loss as loss
import torch.nn as nn

importlib.reload(models)
importlib.reload(training)
importlib.reload(architecture)
importlib.reload(loss)


class LinearModel(nn.Module):
    def __init__(self, inputSize, outputSize, xdos, device):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(inputSize, outputSize, bias = False)
        self.xdos = xdos
        self.device = device
        #self.alignment = torch.zeros(train_size, device = self.device)
        self.to(self.device)

    def forward(self, x):
        """
        Performs the transformations to the features based on the model
        
        Args:
            x (tensor): input features
        
        Returns:
            tensor: output
        """
        out = self.linear(x)
        return out



In [27]:
soap_model = LinearModel(448, 778, xdos, 'cpu')

In [30]:
lr = 0.001
n_epochs = 100000

opt = torch.optim.Adam(soap_model.parameters(), lr = lr, weight_decay = 0)
threshold = 1000
scheduler_threshold = 100
tol = 1e-7
        
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor = 0.1, patience = scheduler_threshold)#0.5)
best_state = copy.deepcopy(soap_model.state_dict())
lowest_loss = torch.tensor(9999)
pred_loss = torch.tensor(0)
trigger = 0
loss_history =[]
pbar = tqdm(range(n_epochs))
for epoch in pbar:
    pbar.set_description(f"Epoch: {epoch}")
    pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)

    for x_data, y_data in train_dataloader_soap:
        opt.zero_grad()
        x_data, y_data = x_data.to(soap_model.device), y_data.to(soap_model.device)
        pred = soap_model.forward(x_data)
        pred_loss = t_get_mse(pred, y_data)#, self.xdos, perc = True)
        new_loss = 1E7 * pred_loss
        new_loss.backward()
        opt.step()
        if pred_loss >100000 or (pred_loss.isnan().any()) :
            print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
            soap_model.load_state_dict(best_state)
            opt = torch.optim.Adam(soap_model.parameters(), lr = lr, weight_decay = 0)


    with torch.no_grad():
        total_pred = soap_model.forward(train_dataloader_soap.dataset.tensors[0])
        total_loss = t_get_mse(total_pred, train_dataloader_soap.dataset.tensors[1])
        rel_loss = t_get_rmse(total_pred, train_dataloader_soap.dataset.tensors[1], soap_model.xdos, perc = True)
        new_loss = total_loss
        if lowest_loss - new_loss > tol: #threshold to stop training
            best_state = copy.deepcopy(soap_model.state_dict())
            lowest_loss = new_loss
            trigger = 0
        else:
            trigger +=1

        if trigger > threshold:
            soap_model.load_state_dict(best_state)
            print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
            break
        if epoch %1000 == 1:
            loss_history.append(lowest_loss.item())





Epoch: 21873:  22%|████████████████████▌                                                                         | 21873/100000 [3:02:52<10:53:10,  1.99it/s, lowest_loss=4.79e-5, pred_loss=2.96e-5, trigger=1000]

Implemented early stopping with lowest_loss: 4.7857860638692426e-05


In [31]:
with torch.no_grad():
    soap_preds = soap_model(soap_total)
    RMSES = evaluator.GetTargetRMSE(soap_preds, "pw", train_index, test_index)
    print (RMSES)

[ 7.34178037 13.08973676  8.87244966  0.04317491  0.07972232  0.05256665]


In [32]:
kernel_model = LinearModel(1000, 778, xdos, 'cpu')

In [33]:
lr = 0.001
n_epochs = 100000

opt = torch.optim.Adam(kernel_model.parameters(), lr = lr, weight_decay = 0)
threshold = 1000
scheduler_threshold = 100
tol = 1e-4
        
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor = 0.1, patience = scheduler_threshold)#0.5)
best_state = copy.deepcopy(kernel_model.state_dict())
lowest_loss = torch.tensor(9999)
pred_loss = torch.tensor(0)
trigger = 0
loss_history =[]
pbar = tqdm(range(n_epochs))
for epoch in pbar:
    pbar.set_description(f"Epoch: {epoch}")
    pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)

    for x_data, y_data in train_dataloader_kernel:
        opt.zero_grad()
        x_data, y_data = x_data.to(kernel_model.device), y_data.to(kernel_model.device)
        pred = kernel_model.forward(x_data)
        pred_loss = t_get_mse(pred, y_data)#, self.xdos, perc = True)
        new_loss = 1E7 * pred_loss
        new_loss.backward()
        opt.step()
        if pred_loss >100000 or (pred_loss.isnan().any()) :
            print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
            kernel_model.load_state_dict(best_state)
            opt = torch.optim.Adam(kernel_model.parameters(), lr = lr, weight_decay = 0)


    with torch.no_grad():
        total_pred = kernel_model.forward(train_dataloader_kernel.dataset.tensors[0])
        total_loss = t_get_mse(total_pred, train_dataloader_kernel.dataset.tensors[1])
        rel_loss = t_get_rmse(total_pred, train_dataloader_kernel.dataset.tensors[1], kernel_model.xdos, perc = True)
        new_loss = total_loss
        if lowest_loss - new_loss > tol: #threshold to stop training
            best_state = copy.deepcopy(kernel_model.state_dict())
            lowest_loss = new_loss
            trigger = 0
        else:
            trigger +=1

        if trigger > threshold:
            kernel_model.load_state_dict(best_state)
            print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
            break
        if epoch %1000 == 1:
            loss_history.append(lowest_loss.item())





Epoch: 2339:   2%|██▏                                                                                             | 2339/100000 [29:24<20:27:36,  1.33it/s, lowest_loss=0.000539, pred_loss=0.000456, trigger=1000]

Implemented early stopping with lowest_loss: 0.0005390271828621282


In [34]:
with torch.no_grad():
    kernel_preds = kernel_model(kernel_total)
    Kernel_RMSES = evaluator.GetTargetRMSE(kernel_preds, "pw", train_index, test_index)
    print (Kernel_RMSES)

[24.6393655  23.17641564 24.33127686  0.14489706  0.14115468  0.14415564]
